In [ ]:
# dependencies

import datetime

import numpy as np
import pandas as pd
import pystan as stan
import matplotlib.pyplot as plt
import seaborn as sns

import xfx.mvglm.cmult
import xfx.misc.plot

In [ ]:
# helper functions

def package_gibbs_samples(samples, factor_names, response_names, meta):

    rfx_samples, prec_samples = zip(*samples)
    rfx_samples = [np.array(samples_) for samples_ in zip(*rfx_samples)]
    prec_samples = np.trace(prec_samples, axis1=2, axis2=3).T
    return package_rfx_samples(rfx_samples, ['_const'] + factor_names, response_names, meta), package_prec_samples(prec_samples, factor_names, meta)

def package_nuts_samples(samples, data, factor_names, response_names, meta):

    alp0_samples = samples['intercept'][:, 0, :-1]
    alp_samples = [[alp_[:, lo:(hi+1)].T for lo, hi in zip(data['lo'] - 1, data['hi'] - 1)] for alp_ in samples['coefs'][:, 0, :-1]]
    rfx_samples = [np.array(samples_) for samples_ in zip(*[[alp0_[np.newaxis]] + alp_ for alp0_, alp_ in zip(alp0_samples, alp_samples)])]
    prec_samples = np.array([[np.trace(np.linalg.inv(tau__)) for tau__ in tau_] for tau_ in samples['cov_factor'][:, 0]]).T
    return package_rfx_samples(rfx_samples, ['_const'] + factor_names, response_names, meta), package_prec_samples(prec_samples, factor_names, meta)

def package_rfx_samples(rfx_samples, factor_names, response_names, meta):

    dfs = []
    for i, (samples_, factor_name) in enumerate(zip(rfx_samples, factor_names)):
        for j in range(samples_.shape[1]):
            df_ = pd.DataFrame(samples_[:, j].T, index=response_names)
            df_.index = df_.index.rename('response')
            df_.columns = df_.columns.rename('iter')
            df_['factor'] = factor_name
            df_['level'] = j
            for k, v in meta.items():
                df_[k] = v
            dfs.append(df_)
    df = pd.concat(dfs).reset_index().set_index(['factor', 'level', 'response'] + list(meta.keys()))
    return df

def package_prec_samples(prec_samples, factor_names, meta):

    df = pd.DataFrame(prec_samples, index=factor_names)
    df.index = df.index.rename('factor')
    df.columns = df.columns.rename('iter')
    for k, v in meta.items():
        df[k] = v
    df = df.reset_index().set_index(['factor'] + list(meta.keys()))
    return df

def est_acf(samples, n_lags):

    acf = samples.apply(lambda x: xfx.misc.plot.est_acf(x.values, n_lags), 1, False, 'expand')
    acf.columns = acf.columns.rename('lag')
    return acf

def est_ess(acfs, titer):
    
    df = pd.DataFrame(index=acfs.index)
    df['iat[iter]'] = acfs.apply(lambda x: xfx.misc.plot.est_int_autocor(x.values), 1, False, 'expand').rename('iat')
    df['iat[sec]'] = df['iat[iter]'] * titer
    df['rate[iter]'] = 1 / (2 * df['iat[iter]'])
    df['rate[sec]'] = df['rate[iter]'] / titer
    return df

In [ ]:
# config

factor_names = ['province_id', 'activity', 'age', 'education', 'municipality_size', 'voting_recall', 'gender']
response_names = ['conservatives', 'social_democrats']
exclude = ['abstention', 'invalid']
seed = 0

In [ ]:
# construct common inputs

cis = pd.read_csv('paper/data/cis.csv')
cis = cis.loc[(cis.study_id == '2019-11-10') & (~cis.voting_intention.isin(exclude)) & (~cis.voting_intention.isna())]
cis['response'] = np.where(cis.voting_intention.isin(response_names), cis.voting_intention, '_others')
cis['voting_recall'] = np.where(cis.voting_recall.isin(response_names), cis.voting_recall, '_others')
cis = cis[factor_names + ['response']].dropna()
codes = cis.apply(lambda x: x.astype('category').cat.codes)
response = pd.get_dummies(codes.response)
indices = codes.drop('response', 1)
n_levels = np.max(indices, 0).astype(np.int64) + 1
rng = np.random.default_rng(seed)

In [ ]:
# construct nuts inputs

counts = codes.groupby(codes.columns.tolist()).size().unstack('response').fillna(0)
coef_idx = counts.index.to_frame() + np.hstack([0, np.cumsum(n_levels[:-1])]) + 1
nuts_inputs = {
    'n_strata': counts.shape[0],
    'n_parties': counts.shape[1],
    'n_factors': len(n_levels),
    'n_coefs': n_levels.sum(),
    'prior_df': counts.shape[1] - 1,
    'lo': np.hstack([0, np.cumsum(n_levels[:-1])]) + 1,
    'hi': np.cumsum(n_levels).values,
    'coef_idx': coef_idx.values,
    'counts': counts.applymap(int).values}

In [ ]:
# sample stan

nuts_n_samples = 1000
stan_model = stan.StanModel('paper/stan/xfx_mvlogit2.stan')
t0 = datetime.datetime.now()
nuts_sampler = stan_model.sampling(data=nuts_inputs, iter=2*nuts_n_samples, warmup=nuts_n_samples, chains=1, seed=0)
t1 = datetime.datetime.now()
nuts_titer = (t1 - t0).total_seconds() / nuts_n_samples
nuts_leaps = nuts_sampler.get_sampler_params(inc_warmup=False)[0]['n_leapfrog__'].mean()
nuts_samples = nuts_sampler.extract(['intercept', 'coefs', 'cov_factor'], permuted=False)

In [ ]:
# compute nuts summaries

nuts_rfx_samples, nuts_prec_samples = package_nuts_samples(nuts_samples, nuts_inputs, factor_names, response_names, {'algo': 'NUTS'})
nuts_rfx_acf, nuts_prec_acf = (est_acf(samples_, 64) for samples_ in (nuts_rfx_samples, nuts_prec_samples))
nuts_rfx_ess, nuts_prec_ess = (est_ess(acfs_, nuts_titer) for acfs_ in (nuts_rfx_acf, nuts_prec_acf))

In [ ]:
# construct gibbs inputs

prior_n_tau = np.repeat(len(response_names), len(n_levels))
gibbs_inputs = (response.values, n_levels.values, indices.values)

In [ ]:
# sample lc

lc_n_samples = 10000
lc_sampler = xfx.mvglm.cmult.sample_posterior(*gibbs_inputs, ome=rng)
next(lc_sampler)
t0 = datetime.datetime.now()
lc_samples = [next(lc_sampler) for _ in range(2 * lc_n_samples)][lc_n_samples:]
t1 = datetime.datetime.now()
lc_titer = (t1 - t0).total_seconds() / lc_n_samples

In [ ]:
# sample vanilla

van_n_samples = 10000
van_sampler = xfx.mvglm.cmult.sample_posterior(*gibbs_inputs, ome=rng, collapse=False)
next(lc_sampler)
t0 = datetime.datetime.now()
van_samples = [next(van_sampler) for _ in range(2 * van_n_samples)][van_n_samples:]
t1 = datetime.datetime.now()
van_titer = (t1 - t0).total_seconds() / van_n_samples

In [ ]:
# compute lc summaries

lc_rfx_samples, lc_prec_samples = package_gibbs_samples(lc_samples, factor_names, response_names, {'algo': 'const/LC'})
lc_rfx_acf, lc_prec_acf = (est_acf(samples_, 256) for samples_ in (lc_rfx_samples, lc_prec_samples))
lc_rfx_ess, lc_prec_ess = (est_ess(acfs_, lc_titer) for acfs_ in (lc_rfx_acf, lc_prec_acf))

In [ ]:
# compute vanilla summaries

van_rfx_samples, van_prec_samples = package_gibbs_samples(van_samples, factor_names, response_names, {'algo': 'const/vGS'})
van_rfx_acf, van_prec_acf = (est_acf(samples_, 256) for samples_ in (van_rfx_samples, van_prec_samples))
van_rfx_ess, van_prec_ess = (est_ess(acfs_, van_titer) for acfs_ in (van_rfx_acf, van_prec_acf))

In [ ]:
# stack

rfx_acf = pd.concat([lc_rfx_acf, van_rfx_acf, nuts_rfx_acf])
prec_acf = pd.concat([lc_prec_acf, van_prec_acf, nuts_prec_acf])
rfx_ess = pd.concat([lc_rfx_ess, van_rfx_ess, nuts_rfx_ess])
prec_ess = pd.concat([lc_prec_ess, van_prec_ess, nuts_prec_ess])

In [ ]:
acf = pd.concat([rfx_acf, prec_acf]).reset_index().melt(id_vars=['algo', 'factor', 'level', 'response'], var_name='lag')
acf['level'] = acf.level.fillna(0)
acf['factor'] = acf.factor.astype('category').cat.codes
acf['group'] = (acf.factor.astype('str') + '-' + acf.level.astype('str')).astype('category').cat.codes
acf['time'] = acf.lag.astype(int) * np.select([acf.algo == 'const/LC-MwG', acf.algo == 'const/Van-MwG'], [lc_titer, van_titer], nuts_titer)
ess = pd.concat([rfx_ess, prec_ess]).reset_index()
ess['level'] = ess.level.fillna(0)

f, axes = plt.subplots(1, 2, figsize=(2 * (8/5 + 4/3), 2), gridspec_kw={'width_ratios': [8/5, 4/3]})
g = sns.lineplot(data=acf, x='time', y='value', hue='algo', style='group', dashes=False, markers=False, legend=False, ci=None, alpha=1/3, lw=1/3, ax=axes[0], hue_order=('const/LC-MwG', 'NUTS', 'const/Van-MwG'))
g.set(xlabel='wall time [sec]', ylabel='ACF', xlim=(-1, 21))
g = sns.boxplot(data=ess, y='algo', x='rate[sec]', order=('const/LC-MwG', 'NUTS', 'const/Van-MwG'), linewidth=1, fliersize=1, sym='o', ax=axes[1])
g.set(ylabel='$\\quad$', xlabel='ESS/sec', xscale='log')